### Graph based line merging

In [ ]:
import networkit as nk
import geopandas as gpd
import shapely
import numpy as np
from shapely.geometry import Point, LineString

ANGLE_TOLERANCE = np.pi/10
TURN_ANGLE_TOLERANCE = np.pi * 0.5  # (little bigger than right angle)

In [ ]:
in_file = r"I:\BERATools\Surmont_New_AOI\Developement\seed_lines_2022.shp"
out_file = r"I:\BERATools\Surmont_New_AOI\Developement\seed_lines_2022_grouped.shp"

# in_file = r"~/BERATools/Surmont_New_AOI/seed_lines_2022.shp"
# out_file = r"~/BERATools/Surmont_New_AOI/seed_lines_2022_grouped.shp"


In [ ]:
data = gpd.read_file(in_file)
data.head(5)


In [ ]:
### remove empty and null geometry
data = data[~data.geometry.isna() & ~data.geometry.is_empty]

In [ ]:
line = data.iloc[[1]]
type(line)

In [ ]:
sindex = data.sindex

In [ ]:
G = nk.Graph(len(data))
G.numberOfNodes()

In [ ]:
def points_in_line(line):
    point_list = []
    try:
        for point in list(line.coords):  # loops through every point in a line
            # loops through every vertex of every segment
            if point:  # adds all the vertices to segment_list, which creates an array
                point_list.append(Point(point[0], point[1]))
    except Exception as e:
        print(e)

    return point_list

def get_angle(line, end_index):
    """
    Calculate the angle of the first or last segment
    line: LineString
    end_index: 0 or -1 of the line vertices. Consider the multipart.
    """
    pts = points_in_line(line)

    if end_index == 0:
        pt_1 = pts[0]
        pt_2 = pts[1]
    elif end_index == -1:
        pt_1 = pts[-1]
        pt_2 = pts[-2]

    delta_x = pt_2.x - pt_1.x
    delta_y = pt_2.y - pt_1.y
    # if np.isclose(pt_1.x, pt_2.x):
    #     angle = np.pi / 2
    #     if delta_y > 0:
    #         angle = np.pi / 2
    #     elif delta_y < 0:
    #         angle = -np.pi / 2
    # else:
    #     angle = np.arctan(delta_y / delta_x)
    angle = np.arctan2(delta_y, delta_x)

    # # arctan is in range [-pi/2, pi/2], regulate all angles to [[-pi/2, 3*pi/2]]
    # if delta_x < 0:
    #     angle += np.pi  # the second or fourth quadrant

    return angle

class VertexNode:
    def __init__(self, line, vertex_index, id) -> None:
        self.vertex = None
        self.line_list = []  # list of dict {'line': line, 'index': 0 or -1, 'id': number}
        self.line_connected = []  # pairs of lines connected

        if line:
            self.add_line(line, vertex_index, id)

    def set_vertex(self, line, vertex_index):
        self.vertex = shapely.force_2d(shapely.get_point(line, vertex_index))
    
    def add_line(self, line, vertex_index, id):
        self.line_list.append({'line': line, 'index': vertex_index, 'id': id})
        self.set_vertex(line, vertex_index)

    def merge(self, vertex):
        self.add_line(vertex.line_list[0]['line'], vertex.line_list[0]['index'], vertex.line_list[0]['id'])

    def get_direction(line, vertex_index):
        pass

    # generate connectivity of all lines
    def check_connectivity(self):
        if len(self.line_list) == 1:
            return

        # if there are 3 and more lines
        angles = [get_angle(i['line'], i['index']) for i in self.line_list]
        angle_visited = [False]*len(angles)

        if len(self.line_list) == 2:
            angle_diff = abs(angles[0] - angles[1])
            angle_diff = angle_diff if angle_diff <= np.pi else angle_diff-np.pi

            #if angle_diff >= TURN_ANGLE_TOLERANCE:
            self.line_connected.append((self.line_list[0]['id'], self.line_list[1]['id']))
            return

        for i, angle_1 in enumerate(angles):
            for j, angle_2 in enumerate(angles[i+1:]):
                if not angle_visited[i+j+1]:
                    angle_diff = abs(angle_1 - angle_2)
                    angle_diff = angle_diff if angle_diff <= np.pi else angle_diff-np.pi
                    if angle_diff < ANGLE_TOLERANCE or np.pi-ANGLE_TOLERANCE < abs(angle_1-angle_2) < np.pi+ANGLE_TOLERANCE:
                        angle_visited[j+i+1] = True  # tenth of PI
                        self.line_connected.append((self.line_list[i]['id'], self.line_list[i+j+1]['id']))


In [ ]:
vertex_list = []
for i, geom in enumerate(data.geometry):
    vertex_list.append(VertexNode(geom, 0, i))
    vertex_list.append(VertexNode(geom, -1, i))

In [ ]:
len(vertex_list)

### merge vertices

In [ ]:
v_points = []
for i in vertex_list:
    v_points.append(i.vertex.buffer(1))  # small polygon around vertices

In [ ]:
v_index = shapely.STRtree(v_points)
merged_vertex_list = []
vertex_visited = [False]*len(vertex_list)

for i, pt in enumerate(v_points):
    if vertex_visited[i]:
        continue

    s_list = v_index.query(pt)

    vertex = vertex_list[i]
    if len(s_list) > 1:
        for j in s_list:
            if j != i:
                vertex.merge(vertex_list[j])
                vertex_visited[j] = True

    merged_vertex_list.append(vertex)
    vertex_visited[i] = True

In [ ]:
len(merged_vertex_list)

In [ ]:
for i in merged_vertex_list:
    i.check_connectivity()

In [ ]:
sum = 0
for i in merged_vertex_list:
    if i.line_connected:
        # print(i.line_connected)
        for edge in i.line_connected:
            G.addEdge(edge[0], edge[1])

In [ ]:
G.numberOfEdges()

In [ ]:
cc = nk.components.ConnectedComponents(G)
cc.run()

In [ ]:
print("number of components ", cc.numberOfComponents())

In [ ]:
groups = [0]*G.numberOfNodes()
group = 0
for i in range(cc.numberOfComponents()):
    component = cc.getComponents()[i]
    for id in component:
        groups[id] = group

    group += 1
        
    # print(group)

In [ ]:
print(groups)

In [ ]:
data['group'] = groups

In [ ]:
data.tail(3)

In [ ]:
data.to_file(out_file)